In [2]:
#문서 분류
import nltk
import random
import feedparser

In [3]:
urls = {
    'mlb': 'https://sports.yahoo.com/mlb/rss.xml',
    'nfl': 'https://sports.yahoo.com/nfl/rss.xml',
}

In [4]:
feedmap = {}
stopwords = nltk.corpus.stopwords.words('english') #불용어 제거

In [5]:
#던어 목록을 가져와 Dictionary에 추가
def featureExtractor(words):
    features = {}
    for word in words:
        if word not in stopwords:
            features["word({})".format(word)] = True
    return features

In [6]:
sentences = []

In [7]:

for category in urls.keys():
    feedmap[category] = feedparser.parse(urls[category])
    print("downloading {}".format(urls[category]))
    for entry in feedmap[category]['entries']:
        data = entry['summary']
        words = data.split()
        sentences.append((category, words))

featuresets = [(featureExtractor(words), category) for category, words in sentences]
random.shuffle(featuresets)

downloading https://sports.yahoo.com/mlb/rss.xml
downloading https://sports.yahoo.com/nfl/rss.xml


In [8]:
total = len(featuresets)
off = int(total/2)
trainset = featuresets[off:]
testset = featuresets[:off]

classifier = nltk.NaiveBayesClassifier.train(trainset)

In [9]:
print(nltk.classify.accuracy(classifier, testset))

0.9019607843137255


In [10]:
classifier.show_most_informative_features(5)
for (i, entry) in enumerate(feedmap['nfl']['entries']):
    if i < 4:
        features = featureExtractor(entry['title'].split())
        category = classifier.classify(features)
        print('{} -> {}'.format(category, entry['summary']))

Most Informative Features
             word(first) = True              mlb : nfl    =      3.6 : 1.0
            word(Monday) = True              nfl : mlb    =      2.8 : 1.0
               word(Tom) = True              nfl : mlb    =      2.0 : 1.0
              word(five) = True              nfl : mlb    =      2.0 : 1.0
              word(pick) = True              nfl : mlb    =      2.0 : 1.0
mlb -> Tony Romo, CBS's top NFL commentator, suggested that the fan who got Tom Brady's 600th TD ball ask for "a date with Gisele." It was crude and unacceptable.
nfl -> Tom Brady might have some explaining to do when he sees the defensive players next at the Tampa Bay Buccaneers' headquarters.
mlb -> The 49ers are sliding, and their coach is under pressure to turn it around.
nfl -> NFC teams lock down top five spots, but AFC clubs are slotted sixth through 11th as power dynamics shift.
